In [1]:
from HARK.utilities import CRRAutility, CRRAutilityP
import numpy as np

CRRAutilityP_hack = lambda c, gam: float('inf') if c == 0.0 else CRRAutilityP(c, gam)

In [2]:
## Doing this because of the CRRAutility warnings

import warnings
warnings.filterwarnings('ignore')

### Consumption with Independent Shocks

_Notebook by Sebastian Benthall_

In this notebook, we will use the generic Bellman stage tools to build and solve a household consumption model with independent shocks (called elsewhere in HARK `ConsIndShock`).

We have predefined some stage definitions in this module:

In [3]:
import cons_stages

CRRA = cons_stages.CRRA
epsilon = cons_stages.epsilon

In [4]:
def display_stage(stage):
    print(f"x: {stage.inputs}, k: {stage.shocks}, a: {stage.actions}, y: {stage.outputs}")

The consumption stage models the transition $a = m - c$ where $c$ is the choice of consumption. It has not shocks.

In [5]:
c_stage = cons_stages.consumption_stage

display_stage(c_stage)

x: ['m'], k: {}, a: ['c'], y: ['a']


In [6]:


def c_v_y_der_terminal(y):
    return CRRAutilityP_hack(y['a'], CRRA)

sol = c_stage.solve(
    y_grid = {'a' : np.linspace(epsilon, 10,50)},
    v_y_der = c_v_y_der_terminal,
    policy_finder_method = 'egm'
)



In [7]:
sol

<xarray.Dataset>
Dimensions:  (m: 51)
Coordinates:
  * m        (m) float64 0.0 0.0002008 0.41 0.8199 ... 18.85 19.26 19.67 20.08
Data variables:
    v_x_der  (m) float64 inf 9.6e+19 2.705e+03 ... 1.182e-05 1.064e-05 9.6e-06
    pi*      (m) float64 0.0 0.0001008 0.2059 0.4116 ... 9.465 9.67 9.876 10.08

The labor stage has no actions. In this stage, the agent experiences exogenous shocks and grows their resources.

In [8]:
l_stage = cons_stages.labor_stage

display_stage(l_stage)

x: ['b'], k: {'theta': <HARK.distribution.Lognormal object at 0x7f49dfb7cf10>, 'psi': <HARK.distribution.Lognormal object at 0x7f49dfb7cfa0>}, a: [], y: ['m']


We can repeat these stages again and again to simulate the household earning and consuming over time.

But there's one small problem: the output of the consumption stage is $m$ while the input of the labor stage is $b$.

We need to transform $m$ to $b$.

In [9]:
from HARK.stage import Stage, backwards_induction

### TWIST STAGE

def twist(mapper):
    return Stage(
        transition = lambda x, k, a : {mapper[xi] : x[xi] for xi in mapper}, 
        inputs = list(mapper.keys()), 
        actions = [],
        outputs = list(mapper.values())
    )
    
      
twist_stage = twist({'a' : 'b'})



In [10]:
x_space =  np.linspace(epsilon*2,8,50)

stages_data = [
    {
        'stage' : c_stage,
        'x_grid' : {'m' : x_space},
        'optimizer_args' :{
            'a0f' : lambda x: x['m'] - epsilon
        },
        'method' : 'egm'
    },
    {
        'stage' : twist_stage,
        'x_grid' : {'a' : x_space},
        'optimizer_args' :{}
    },
    {
        'stage' : l_stage,
        'x_grid' : {
            'b' : x_space,
        },
        'shock_approx_params' : {
            'psi' : 4, 
            'theta' : 4, 
        },
    }
]

In [12]:
def g_v_y_der_terminal(y):
    return CRRAutilityP_hack(y['a'], CRRA)

sols = backwards_induction(stages_data * 4, g_v_y_der_terminal)

11: X: ['b'], K: ['theta', 'psi'], A: [], Y: ['m']
Grid size: 800


TypeError: interp only works for a numeric type array. Given object.

In [ ]:
import matplotlib.pyplot as plt

for i, s in enumerate([sols[i] for i in (range(0,12, 3))]):
    plt.plot(x_space, [s.pi_star({'m' : m,}, {})['c'] for m in x_space], label = f"{i}")
    
plt.legend()